In [1]:
import sys 
sys.path.append("../..")
sys.path.append("..")

from importlib import reload
from tqdm import tqdm

import joseph
from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *


reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)

/Users/josephbloom/miniforge3/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/josephbloom/miniforge3/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/josephbloom/miniforge3/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:

model = HookedTransformer.from_pretrained(
    "gpt2-small",
    # "tiny-stories-2L-33M",
    # "attn-only-2l",
    # center_unembed=True,
    # center_writing_weights=True,
    # fold_ln=True,
    # refactor_factored_attn_matrices=True,
    fold_ln=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)


path = "../week_8_jan/artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28/1100001280_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152.pt"
# path = "./artifacts/sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152:v9/final_sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)

print(sparse_autoencoder.cfg)


# sanity check
text = "Many important transition points in the history of science have been moments when science 'zoomed in.' At these points, we develop a visualization or tool that allows us to see the world in a new level of detail, and a new field of science develops to study the world through this lens."
model(text, return_type="loss")

Loaded pretrained model gpt2-small into HookedTransformer
LanguageModelSAERunnerConfig(model_name='gpt2-small', hook_point='blocks.10.hook_resid_pre', hook_point_layer=10, hook_point_head_index=None, dataset_path='Skylion007/openwebtext', is_dataset_tokenized=False, context_size=128, use_cached_activations=False, cached_activations_path='activations/Skylion007_openwebtext/gpt2-small/blocks.10.hook_resid_pre', d_in=768, n_batches_in_buffer=128, total_training_tokens=3000000000, store_batch_size=32, device='mps', seed=42, dtype=torch.float32, b_dec_init_method='geometric_median', expansion_factor=64, from_pretrained_path=None, l1_coefficient=8e-05, lr=0.0012, lr_scheduler_name=None, lr_warm_up_steps=5000, train_batch_size=4096, feature_sampling_window=1500, feature_sampling_method='anthropic', resample_batches=1028, feature_reinit_scale=0.2, dead_feature_window=50000, dead_feature_estimation_method='no_fire', dead_feature_threshold=1e-06, log_to_wandb=True, wandb_project='mats_sae_traini

tensor(3.3225, device='mps:0')

In [13]:
prompt = """
Putting Cats, Dogs, Mice in alphabetical order, we get:
1. Cats 2. Dogs 3. Mice
Putting Carrot, Apple, Banana in alphabetical order, we get:
1."""
answer = " Carrot"
utils.test_prompt(prompt, answer, model)

Tokenized prompt: ['<|endoftext|>', '\n', 'Putting', ' Cats', ',', ' Dogs', ',', ' M', 'ice', ' in', ' alphabet', 'ical', ' order', ',', ' we', ' get', ':', '\n', '1', '.', ' Cats', ' 2', '.', ' Dogs', ' 3', '.', ' M', 'ice', '\n', 'Putting', ' Car', 'rot', ',', ' Apple', ',', ' Banana', ' in', ' alphabet', 'ical', ' order', ',', ' we', ' get', ':', '\n', '1', '.']
Tokenized answer: [' Car', 'rot']


Performance on answer token:
Rank: 1        Logit: 16.38 Prob: 20.65% Token: | Car|

Top 0th token. Logit: 17.16 Prob: 44.83% Token: | Cats|
Top 1th token. Logit: 16.38 Prob: 20.65% Token: | Car|
Top 2th token. Logit: 15.51 Prob:  8.61% Token: | Cat|
Top 3th token. Logit: 13.74 Prob:  1.48% Token: | Dogs|
Top 4th token. Logit: 13.44 Prob:  1.10% Token: | C|
Top 5th token. Logit: 12.92 Prob:  0.65% Token: | Birds|
Top 6th token. Logit: 12.72 Prob:  0.53% Token: | Animals|
Top 7th token. Logit: 12.67 Prob:  0.51% Token: | Apple|
Top 8th token. Logit: 12.67 Prob:  0.50% Token: | A|
Top 9th token. Logit: 12.38 Prob:  0.38% Token: | Cars|


Performance on answer token:
Rank: 0        Logit: 22.75 Prob: 71.52% Token: |rot|

Top 0th token. Logit: 22.75 Prob: 71.52% Token: |rot|
Top 1th token. Logit: 21.79 Prob: 27.58% Token: |rots|
Top 2th token. Logit: 17.24 Prob:  0.29% Token: |p|
Top 3th token. Logit: 16.74 Prob:  0.18% Token: |ro|
Top 4th token. Logit: 15.96 Prob:  0.08% Token: |ot|
Top 5th token. Logit: 15.44 Prob:  0.05% Token: |pet|
Top 6th token. Logit: 14.32 Prob:  0.02% Token: |pe|
Top 7th token. Logit: 13.73 Prob:  0.01% Token: |rel|
Top 8th token. Logit: 13.68 Prob:  0.01% Token: |r|
Top 9th token. Logit: 13.43 Prob:  0.01% Token: |ole|


Ranks of the answer tokens: [(' Car', 1), ('rot', 0)]

In [ ]:
import nltk

nltk.download("words")

from nltk.corpus import words


In [55]:
d_vocab = model.tokenizer.vocab_size
vocab_df = pd.DataFrame(
    {
        "token": np.arange(d_vocab),
        "string": model.to_str_tokens(np.arange(d_vocab)),
    }
)
vocab_df["string_lower_stripped"] = vocab_df.string.str.lower().str.strip()
vocab_df["is_alpha"] = vocab_df.string.str.match(r"^( ?)[a-z]+$")
vocab_df["is_word"] = vocab_df.string_lower_stripped.isin(words.words())
vocab_df["is_fragment"] = vocab_df.string.str.match(r"^[a-z]+$")
vocab_df["has_space"] = vocab_df.string.str.match(r"^ [A-Za-z]+$")
vocab_df["num_chars"] = vocab_df.string.apply(lambda n: len(n.strip()))
vocab_df

,token,string,string_lower_stripped,is_alpha,is_word,is_fragment,has_space,num_chars
0,0,!,!,False,False,False,False,1
1,1,"""","""",False,False,False,False,1
2,2,#,#,False,False,False,False,1
3,3,$,$,False,False,False,False,1
4,4,%,%,False,False,False,False,1
...,...,...,...,...,...,...,...,...
50252,50252,regress,regress,True,True,False,True,7
50253,50253,Collider,collider,False,False,False,True,8
50254,50254,informants,informants,True,False,False,True,10
50255,50255,gazed,gazed,True,False,False,True,5


In [60]:
word_df = vocab_df[vocab_df.is_word & vocab_df.has_space & (vocab_df.num_chars == 5)]
word_df.head(30)

,token,string,string_lower_stripped,is_alpha,is_word,is_fragment,has_space,num_chars
511,511,their,their,True,True,False,True,5
543,543,which,which,True,True,False,True,5
546,546,about,about,True,True,False,True,5
561,561,would,would,True,True,False,True,5
584,584,other,other,True,True,False,True,5
612,612,there,there,True,True,False,True,5
706,706,after,after,True,True,False,True,5
714,714,could,could,True,True,False,True,5
717,717,first,first,True,True,False,True,5
739,739,under,under,True,True,False,True,5


In [64]:
def make_single_prompt():
    words = [word_df.string.sample().item().strip() for _ in range(3)]
    print(words)
    sorted_words = sorted(words)
    return f"Question: Put the following words in order: {', '.join(words)}\nAnswer: {', '.join(sorted_words)}"
make_single_prompt()

['siege', 'Spell', 'Bundy']


'Question: Put the following words in order: siege, Spell, Bundy\nAnswer: Bundy, Spell, siege'

In [65]:
import circuitsvis as cv

example_prompt = make_single_prompt()
logits, cache = model.run_with_cache(example_prompt)
display(cv.logits.token_log_probs(model.to_tokens(example_prompt), model(example_prompt)[0].log_softmax(dim=-1), model.to_string))

['GROUP', 'Cheap', 'funky']


In [ ]:
def make_single_prompt():
    word = short_chars_vocab_df.string.sample().item().strip()
    return f" {word}:" + "".join([f" {c.upper()}" for c in word.strip()])


def make_kshot_prompt(k=3):
    return "\n".join([make_single_prompt() for _ in range(k)])


def make_kshot_prompts(n=10, k=3):
    return [make_kshot_prompt(k) for _ in range(n)]


def get_answer_index(prompts):
    batch_size = len(prompts)
    answer_index = torch.zeros((batch_size, 5), device="cuda", dtype=torch.int64) - 1
    for i in range(batch_size):
        for j in range(5):
            answer_index[i, j] = alphabet.index(prompts[i][2 * j - 9].lower())
    return answer_index
